 <table><tr><td bgcolor=#ffccff> <font color=#1a001a size=7 face="black body">This is KDA applied on benchmark dataset </font> </td></tr></table>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 <table><tr><td bgcolor=#99ff99> <font color=#3333ff size=10 face="black body">Preprocessing of dataset </font> </td></tr></table>
 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on %(date)s

@author: %(username)s
"""

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
data=pd.read_csv(r'../input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv')
data.drop(columns=['sessionIndex','rep'],axis=1,inplace=True)
visual_data=data
#One Hot encoding the subject column. seperating the label from the rest of columns
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
unique_cols=data['subject'].nunique()

data=np.array(ct.fit_transform(data))
X=data[:,unique_cols:]
y=data[:,:unique_cols]
print(y.shape)


#splitting the dataset into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
#training:validation:test=70:0:30

#applying feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)

X_test=sc.transform(X_test)



In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
visual_data=pd.read_csv(r'../input/keystroke-dynamics-benchmark-data-set/DSL-StrongPasswordData.csv')
visual_data.drop(columns=['sessionIndex','rep'],axis=1,inplace=True)


 <table><tr><td bgcolor=#ffffcc> <font color=#9900cc size=10 face="black body">Training Model </font> </td></tr></table>

In [ ]:
import tensorflow as tf
cnn=tf.keras.models.Sequential()

#cnn.add(tf.keras.layers.Flatten())
#
'''cnn.add(tf.keras.layers.Dense(
units=256,activation='relu'
))'''
cnn.add(tf.keras.layers.Dense(
units=256,activation='relu'
))

cnn.add(BatchNormalization())

cnn.add(tf.keras.layers.Dense(
units=512,activation='relu'
))

cnn.add(BatchNormalization())

cnn.add(tf.keras.layers.Dense(
units=1024,activation='relu'
))

cnn.add(BatchNormalization())

cnn.add(tf.keras.layers.Dense(
units=512,activation='relu'
))
cnn.add(BatchNormalization())

cnn.add(tf.keras.layers.Dense(
    units=unique_cols,activation='sigmoid'
))

cnn.compile(
optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

batch_size = 32


epochs=50

In [ ]:
import datetime
%load_ext tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#callback=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',model='auto',patience=15,verbose=1,restore_best_weights=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


hist=cnn.fit(X_train, y_train, epochs = epochs, batch_size=batch_size, validation_split = 0.0, verbose = 1)


In [ ]:
print(cnn.summary())



In [ ]:
#print(hist.history['loss'])

score,acc = cnn.evaluate(X_test, y_test, verbose = 1, batch_size = batch_size)
print("score: "+str(score)+" accuracy: "+str(acc))

 <table><tr><td bgcolor=#ffff99> <font color=#b30059 size=10 face="black body">Insights into Training the Dataset </font> </td></tr></table>


In [ ]:
train_loss = hist.history['loss']
#val_loss   = hist.history['val_loss']
acc = hist.history['accuracy']
#val_acc = hist.history['val_accuracy']


In [ ]:
plt.plot(range(1,epochs+1),train_loss)
#plt.plot(range(1,epochs+1),val_loss)
plt.title('Losses vs Epochs')
plt.xlabel('Epoch Number')
plt.ylabel('Losses')
plt.legend(["Training loss"])
#plt.legend(["Training loss", "Validation loss"])

In [ ]:
plt.plot(range(1,epochs+1),acc)
#plt.plot(range(1,epochs+1),val_acc)
plt.title('Accuracies vs Epochs')
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.legend(["Training accuracy"]) 
#plt.legend(["Training accuracy", "Validation accuracy"])

In [ ]:
y_pred=cnn.predict(X_test)
'''as the softmax outputs probability of each class there is need for conversion from  probability to 0s and 1s
where the class corresponding to highest probability will be set to 1 and the rest to 0s.
y_pred is an array of probabilities where y_pred_binary is an array of 0s and 1s.'''
y_pred_binary=np.zeros(shape=y_pred.shape)
examples=y_pred.shape[0]
print(examples)
for i in range(examples):
    index=np.argmax(y_pred[i])
    y_pred_binary[i][index]=1


In [ ]:
data

 # <table><tr><td bgcolor=#b3ffff> <font color= #990099 size=10 face="black body">ROC Curve</font> </td></tr></table>


In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def plotROC():
        
    predictions = cnn.predict(X_test)
    
    fpr, tpr, thresholds = roc_curve(y_test.flatten(), predictions.flatten())
    eer = brentq(lambda x : 1.0 - x - interp1d(fpr, tpr)(x), 0.0, 1.0)
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='AUC for {:5s} = {:.3f}, EER = {:.3f}'.format(
        "Multiclass FCNN", auc(fpr, tpr), eer))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')

plotROC()

# <table><tr><td bgcolor=#99ff99> <font color= #0080ff size=10 face="black body">**Data Visualization**</font> </td></tr></table>




In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sn # statistical graphs
import matplotlib.pyplot as plt # data visualization / graphs
plt.style.use('seaborn-dark')


In [ ]:
visual_data.iloc[:,:-1]

# <table><tr><td bgcolor=#99ff99> <font color= #0080ff size=10 face="black body">Correlation Plot</font> </td></tr></table>

In [ ]:
corrMatrix = visual_data[['H.period', 'DD.period.t',
       'UD.period.t', 'H.t', 'DD.t.i', 'UD.t.i', 'H.i', 'DD.i.e', 'UD.i.e',
       'H.e', 'DD.e.five', 'UD.e.five', 'H.five', 'DD.five.Shift.r',
       'UD.five.Shift.r', 'H.Shift.r', 'DD.Shift.r.o', 'UD.Shift.r.o', 'H.o',
       'DD.o.a', 'UD.o.a', 'H.a', 'DD.a.n', 'UD.a.n', 'H.n', 'DD.n.l',
       'UD.n.l', 'H.l', 'DD.l.Return', 'UD.l.Return', 'H.Return']].corr()
fig, ax = plt.subplots(figsize=(10,10))
sn.heatmap(corrMatrix, linewidths=0.1, cmap="BuPu")
plt.show()

In [ ]:
visual_data.columns
h_cols=['subject']
ud_cols=['subject']
dd_cols=['subject']
uu_cols=['subject']
for column in visual_data.columns:
    if 'H' in column:
        
        h_cols.append(str(column))
    if 'UD' in column:
        ud_cols.append(str(column))
    if 'DD' in column:
        dd_cols.append(str(column))
    if 'UU' in column:
        uu_cols.append(str(column))    
print(dd_cols)
print(uu_cols)
print(h_cols)
print(ud_cols)



# <table><tr><td bgcolor=#ffb3d1> <font color= #0080ff size=10 face="black body"> Average Inter subject variation</font> </td></tr></table>



# <table><tr><td bgcolor=#ffb3d1> <font color=#0066ff size=10 face="black body">1.H Plots</font> </td></tr></table>



In [ ]:
keyHoldLatency = visual_data[h_cols]
keyHoldLatency = keyHoldLatency.where((keyHoldLatency['subject']=='s002') | (keyHoldLatency['subject']=='s003') | (keyHoldLatency['subject']=='s004') | (keyHoldLatency['subject']=='s005') | (keyHoldLatency['subject']=='s007') | (keyHoldLatency['subject']=='s008'))
keyHoldLatency = keyHoldLatency.groupby('subject').agg('mean')
keyHoldLatency.T.plot(figsize=(8,5))

# <table><tr><td bgcolor=#ffb3d1> <font color=#ff1a1a size=10 face="black body">2.UD Plots</font> </td></tr></table>




In [ ]:
keyUDLatency = visual_data[ud_cols]
keyUDLatency = keyUDLatency.where((keyUDLatency['subject']=='s002') | (keyUDLatency['subject']=='s003') | (keyUDLatency['subject']=='s004') | (keyUDLatency['subject']=='s005') | (keyUDLatency['subject']=='s007') | (keyUDLatency['subject']=='s008'))
keyUDLatency = keyUDLatency.groupby('subject').agg('mean')
keyUDLatency.T.plot(figsize=(8,5))

# <table><tr><td bgcolor=#ffb3d1> <font color= #33ccff size=10 face="black body">3.DD Plots</font> </td></tr></table>





In [ ]:
keyDDLatency = visual_data[dd_cols]
keyDDLatency = keyDDLatency.where((keyDDLatency['subject']=='s002') | (keyDDLatency['subject']=='s003') | (keyDDLatency['subject']=='s004') | (keyDDLatency['subject']=='s005') | (keyDDLatency['subject']=='s007') | (keyDDLatency['subject']=='s008'))
keyDDLatency = keyDDLatency.groupby('subject').agg('mean')
keyDDLatency.T.plot(figsize=(8,5))

# <table><tr><td bgcolor=#ffff80> <font color=#800080 size=10 face="black body">Intrasubject variation</font> </td></tr></table>


# <table><tr><td bgcolor=#ffff80> <font color=#0044cc size=10 face="black body">1.H plot</font> </td></tr></table>


In [ ]:
subKeyHoldLatency =visual_data.where((visual_data['subject']=='s021')).dropna().sample(n=12)
subKeyHoldLatency = subKeyHoldLatency[['subject','H.period','H.t','H.i', 'H.e','H.five','H.Shift.r','H.o','H.a','H.n','H.l','H.Return']]
subKeyHoldLatency = subKeyHoldLatency.set_index('subject')
subKeyHoldLatency.T.plot(figsize=(8,5), legend=False)

# <table><tr><td bgcolor=#ffff80> <font color=#0044cc size=10 face="black body">2.UD plot</font> </td></tr></table>




In [ ]:
subKeyUDLatency = visual_data.where((visual_data['subject']=='s021')).dropna().sample(n=12)
subKeyUDLatency = subKeyUDLatency[['subject','UD.period.t','UD.t.i','UD.i.e','UD.e.five','UD.five.Shift.r','UD.Shift.r.o','UD.o.a','UD.a.n','UD.n.l','UD.l.Return']]
subKeyUDLatency = subKeyUDLatency.set_index('subject')
subKeyUDLatency.T.plot(figsize=(8,5), legend=False)

# <table><tr><td bgcolor=#ffff80> <font color=#0044cc size=10 face="black body">3.DD plot</font> </td></tr></table>




In [ ]:
subKeyDDLatency = visual_data.where((visual_data['subject']=='s021')).dropna().sample(n=12)
subKeyDDLatency = subKeyDDLatency[['subject','DD.period.t','DD.t.i','DD.i.e','DD.e.five','DD.five.Shift.r', 'DD.Shift.r.o','DD.o.a','DD.a.n', 'DD.n.l','DD.l.Return']]
subKeyDDLatency = subKeyDDLatency.set_index('subject')
subKeyDDLatency.T.plot(figsize=(8,5), legend=False)